#6.6 RetrievalQA

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA


In [14]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

@traceable
def generate_response(prompt):
    return llm.invoke(prompt)


In [15]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

chain.invoke("Describe Victory Mansions")


{'query': 'Describe Victory Mansions',
 'result': 'Based on the provided text, Victory Mansions is a building with glass doors, hallways that smell of boiled cabbage and old rag mats, and a non-functioning elevator (lift) at least some of the time due to electricity cuts.  Posters depicting a large face of a man with a caption "BIG BROTHER IS WATCHING YOU" are located on each landing.  The building has at least seven flights of stairs.\n'}